<a href="https://colab.research.google.com/github/mojtabaSefidi/Machine-Learning-with-Graphs/blob/main/MLG_Final_Project_100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !unzip ml-100k.zip
# !pip uninstall jupyter
# !pip install jupyter

## Install Essential Packages

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!pip install -q torch_geometric
!pip install -q torch-sparse==0.6.13
!pip install -q torch_scatter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Install Essential Libraries

In [3]:

import torch
import networkx as nx
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.loader import DataLoader
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv, SAGEConv, global_add_pool
from torch_geometric.data import HeteroData
from torch_geometric import transforms
from torch_geometric.loader import LinkNeighborLoader

In [4]:
from numpy.random import seed
from tensorflow import keras
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from torch_geometric.loader import LinkNeighborLoader
import numpy as np
import pandas as pd
import torch
import random

In [6]:
def plot_history(list_train_loss, list_train_acc, list_val_loss, list_val_acc, n_epochs, title):
    
    plt.figure(figsize=(18,8),linewidth = 7, edgecolor="whitesmoke")    
    n = n_epochs
    
    plt.plot(list(range(1, n_epochs+1)), list_train_acc, color='orange',marker=".")
    plt.plot(list(range(1, n_epochs+1)), list_train_loss,'b',marker=".")
    
    plt.plot(list(range(1, n_epochs+1)), list_val_acc,'r')  
    plt.plot(list(range(1, n_epochs+1)), list_val_loss,'g')
    
    plt.legend(['Train Accuracy','Train Loss','Test Accuracy','Test Loss'])
    plt.grid(True)
    
    # plt.gca().set_ylim(0,1)

    plt.xlabel("Number of Epochs")
    plt.ylabel("Value")
    plt.suptitle(title, size=16, y=0.927)
    plt.show()

## Read the Dataset

In [7]:
# from ogb.graphproppred import PygGraphPropPredDataset
# from torch_geometric.loader import DataLoader

# dataset = PygGraphPropPredDataset(name = 'ogbg-molhiv') 

# split_idx = dataset.get_idx_split() 
# train_data_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True)
# valiation_data_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=True)
# test_data_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=True)

In [8]:
# !unzip /content/MovieLens1M.zip

In [9]:
# movies_df = pd.read_csv('/content/ml-100k/u.item', sep='|', engine='python', encoding="latin-1",
#                         names = ['MovieID', 'Movie Title', 'Release Date', 'video release date','IMDbURL',
#                                  'Unknown','Action','Adventure','Animation', 'Childrens','Comedy',
#                                  'Crime','Documentary','Drama','Fantasy', 'Film-Noir','Horror',
#                                  'Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'])

# rating_df = pd.read_csv('/content/ml-100k/u.data', sep='\t', engine='python', encoding="latin-1",
#                         names=['UserID','MovieID','Rating','Timestamp'])

# users_df = pd.read_csv('/content/ml-100k/u.user', sep='|', engine='python', encoding="latin-1",
#                        names=['UserID','Gender','Age','Occupation','Zipcode'])

# movies_df.to_csv('/content/gdrive/MyDrive/MLG_Final_Project/MovieLens100k/Data/movies_df.csv', index=False)
# rating_df.to_csv('/content/gdrive/MyDrive/MLG_Final_Project/MovieLens100k/Data/rating_df.csv', index=False)
# users_df.to_csv('/content/gdrive/MyDrive/MLG_Final_Project/MovieLens100k/Data/users_df.csv', index=False)

In [10]:
# geners = np.array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama',
#                    'Fantasy', 'Film_Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western'], dtype=str)

# values = list(range(0, len(geners)))
# geners2vec = dict(zip(geners, values))

# def extract_geners(text, sep='|'):
#   return text.split(sep)

# def geners2vector(df_geners, maper):
#   result = np.zeros((len(df_geners),len(maper)), dtype='int8')
#   for index, text in enumerate(df_geners):
#     geners = extract_geners(text)
#     for gener in geners:
#       result[index][maper.get(gener)] = 1
  
#   return result

In [11]:
# def year_extractor(text):
#   return text[text.rfind('(')+1:text.rfind(')')]

def add_average_rating(rating_df, movies_df, Movie_id_col='MovieID', rating_col='Rating'):
  
  rating_avg = rating_df.groupby(Movie_id_col).mean()[rating_col].round(4).to_dict()
  result = [] 
  for id in movies_df[Movie_id_col]:
    result.append(rating_avg.get(id, 0))
  movies_df['Average Rating'] = result
  return movies_df

# Occupation_mapper = { 0: "other", 1: "academic/educator", 2: "artist", 3: "clerical/admin",
#                      4: "college/grad student",5: "customer service", 6: "doctor/health care",
#                      7: "executive/managerial", 8: "farmer", 9: "homemaker", 10: "K-12 student",
#                      11: "lawyer", 12: "programmer", 13: "retired", 14: "sales/marketing",
#                      15: "scientist", 16: "self-employed", 17: "technician/engineer",
#                      18: "tradesman/craftsman", 19: "unemployed", 20: "writer"}

# def code2Occupation(occupation_col, mapper):
#   return occupation_col.map(mapper)


# def extract_user_feature(users_df):
#   scaler = StandardScaler()
#   age = scaler.fit_transform(users_df[['Age']])
  
#   encoder = OneHotEncoder(handle_unknown='ignore')
#   occupation = encoder.fit_transform(users_df[['Occupation']]).toarray()
#   features = np.hstack((users_df[['Gender']], age, occupation))
#   return torch.from_numpy(features).to(torch.float)

# def extract_movie_feature(movies_df, mapper):
#   scaler = StandardScaler()
#   numerical = scaler.fit_transform(movies_df[['year',	'averge_rating']])
  
#   categorical = geners2vector(movies_df['Genres'], mapper)
#   features = np.hstack((numerical, categorical))
#   return torch.from_numpy(features).to(torch.float)

def Timestamp2Date(timestamp):
  return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

def add_age_group(df, age_col='Age'):
  bins= [0,20,25,39,60,110]
  labels = ['Teenage','Young Adult','Adult', 'Older Adult','Old']
  df['AgeGroup'] = pd.cut(df[age_col], bins=bins, labels=labels, right=False)
  return df

def find_unknowns(movies_df, title_col='Movie Title', movie_id_col='MovieID'):
  unknown_movies = movies_df[movies_df[title_col]=='unknown'][movie_id_col]
  return unknown_movies

def remove_unknows(df, list_unknown_movies, movie_id_col='MovieID'):
  return df[~df[movie_id_col].isin(list_unknown_movies)]


In [12]:
movies_df = pd.read_csv('/content/gdrive/MyDrive/MLG_Final_Project/MovieLens100k/Data/movies_df.csv')
all_ratings_df = pd.read_csv('/content/gdrive/MyDrive/MLG_Final_Project/MovieLens100k/Data/rating_df.csv')

users_df = pd.read_csv('/content/gdrive/MyDrive/MLG_Final_Project/MovieLens100k/Data/users_df.csv')
movies_df.shape, users_df.shape


((1682, 24), (943, 5))

In [13]:
list_unknown_movies = find_unknowns(movies_df)

In [14]:
def preprocess_movies_df(movies_df, list_unknown_movies, all_ratings_df):
  movies_df = remove_unknows(movies_df, list_unknown_movies)
  movies_df.drop(['video release date', 'IMDbURL'], axis=1, inplace=True)
  movies_df['Release Date'] = pd.to_datetime(movies_df['Release Date'])
  movies_df['Release Year'] = movies_df['Release Date'].dt.year
  movies_df['Release Month'] = movies_df['Release Date'].dt.month
  movies_df['Release Day'] = movies_df['Release Date'].dt.strftime('%j').apply(int)
  movies_df = movies_df.sort_values(by=['Release Date']).reset_index(drop=True)
  movies_df = add_average_rating(all_ratings_df, movies_df)
  return movies_df

movies_df = preprocess_movies_df(movies_df, list_unknown_movies, all_ratings_df)
movies_df.tail(2).T

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-14-db653e20aac5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['Release Date'] = pd.to_datetime(movies_df['Release Date'])
<ipython-input-14-db653e20aac5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

,1679,1680
MovieID,1432,315
Movie Title,"Mighty, The (1998)",Apt Pupil (1998)
Release Date,1998-10-09 00:00:00,1998-10-23 00:00:00
Unknown,0,0
Action,0,0
Adventure,0,0
Animation,0,0
Childrens,0,0
Comedy,0,0
Crime,0,0


In [15]:
users_df = add_age_group(users_df)
users_df

,UserID,Age,Gender,Occupation,Zipcode,AgeGroup
0,1,24,M,technician,85711,Young Adult
1,2,53,F,other,94043,Older Adult
2,3,23,M,writer,32067,Young Adult
3,4,24,M,technician,43537,Young Adult
4,5,33,F,other,15213,Adult
...,...,...,...,...,...,...
938,939,26,F,student,33319,Adult
939,940,32,M,administrator,02215,Adult
940,941,20,M,student,97229,Young Adult
941,942,48,F,librarian,78209,Older Adult


In [16]:
def preprocess_rating_df(ratings_df, list_unknown_movies, extract_details=False):
  ratings_df = remove_unknows(ratings_df, list_unknown_movies)
  if extract_details:
    ratings_df.loc[:,'Timestamp'] = ratings_df.loc[:,'Timestamp'].apply(Timestamp2Date)
    ratings_df.sort_values(by='Timestamp', inplace=True)
    ratings_df.reset_index(drop=True, inplace=True)

    ratings_df.loc[:,'Timestamp'] = pd.to_datetime(ratings_df.loc[:,'Timestamp'])
    ratings_df.loc[:,'Year'] = ratings_df.loc[:,'Timestamp'].dt.year
    ratings_df.loc[:,'Month'] = ratings_df.loc[:,'Timestamp'].dt.month
    ratings_df.loc[:,'Weekday'] = ratings_df.loc[:,'Timestamp'].dt.weekday
    ratings_df.loc[:,'Hour'] = ratings_df.loc[:,'Timestamp'].dt.hour
    ratings_df.loc[:,'DayofYear'] = ratings_df.loc[:,'Timestamp'].dt.strftime('%j')
  
  return ratings_df

In [18]:
def generate_train_validation(path, list_unknown_movies, factor=0.5):
  df = pd.read_csv(path, sep='\t', engine='python', encoding="latin-1",
                          names=['UserID','MovieID','Rating','Timestamp'])

  train_df = pd.DataFrame(columns=['UserID','MovieID','Rating','Timestamp'])
  validation_df = pd.DataFrame(columns=['UserID','MovieID','Rating','Timestamp'])

  for movie_id in df['MovieID'].unique():
    df_temp = df[df['MovieID']== movie_id]
    df_temp = df_temp.sort_values('Timestamp')
    lenght_window = len(df_temp)
    if lenght_window != 1:
      half = round(lenght_window * factor)
      validation_df = pd.concat([validation_df, df_temp.iloc[:half,:]])
      train_df = pd.concat([train_df, df_temp.iloc[half:,:]])
    else: 
      if len(train_df) > len(validation_df):
        validation_df = pd.concat([validation_df, df_temp])
      
      elif len(train_df) < len(validation_df):
        train_df = pd.concat([train_df, df_temp])
      
      else:

        if random.random() >= 0.5:
          train_df = pd.concat([train_df, df_temp])
        else:
          validation_df = pd.concat([validation_df, df_temp])
        
  train_df = preprocess_rating_df(train_df, list_unknown_movies)
  validation_df = preprocess_rating_df(validation_df, list_unknown_movies)
  return train_df.reset_index(drop=True), validation_df.reset_index(drop=True)



In [21]:
def generate_test(path, list_unknown_movies):
   test_df = pd.read_csv(path, sep='\t', engine='python', encoding="latin-1", names=['UserID','MovieID','Rating','Timestamp'])
   test_df = preprocess_rating_df(test_df, list_unknown_movies)
   return test_df.reset_index(drop=True)



In [24]:
train_rating_df, validation_rating_df = generate_train_validation('/content/u1.base', list_unknown_movies)
test_rating_df = generate_test('/content/u1.test', list_unknown_movies)
train_rating_df.shape, validation_rating_df.shape, test_rating_df.shape

((39999, 4), (39998, 4), (19994, 4))

In [26]:
def finalize_dfs(users_df, movies_df,
                 train_rating_df, validation_rating_df, test_rating_df,
                 user_id_col='UserID', movie_id_col='MovieID'
                 ):
  unique_user_id = users_df['UserID'].unique()
  unique_user_id = pd.DataFrame(data={
      'UserID': unique_user_id,
      'mappedID': pd.RangeIndex(len(unique_user_id))
      })
  userid_mapper = dict(zip(unique_user_id.iloc[:,0], unique_user_id.iloc[:,-1]))
  
  # print("1. Mapping UserID to consecutive values... ")

  unique_movie_id = movies_df['MovieID'].unique()
  unique_movie_id = pd.DataFrame(data={
      'MovieID': unique_movie_id,
      'mappedID': pd.RangeIndex(len(unique_movie_id))
      })
  movieid_mapper = dict(zip(unique_movie_id.iloc[:,0], unique_movie_id.iloc[:,-1]))
  # print("2. Mapping MovieID to consecutive values... ")

  users_df['UserID'] = users_df['UserID'].map(userid_mapper)
  users_df = users_df.sort_values(by='UserID').reset_index(drop=True)
  movies_df['MovieID'] =  movies_df['MovieID'].map(movieid_mapper)
  movies_df = movies_df.sort_values(by='MovieID').reset_index(drop=True)
  train_rating_df['UserID'] = train_rating_df['UserID'].map(userid_mapper)
  train_rating_df['MovieID'] =  train_rating_df['MovieID'].map(movieid_mapper)
  validation_rating_df['UserID'] = validation_rating_df['UserID'].map(userid_mapper)
  validation_rating_df['MovieID'] =  validation_rating_df['MovieID'].map(movieid_mapper)
  test_rating_df['UserID'] = test_rating_df['UserID'].map(userid_mapper)
  test_rating_df['MovieID'] =  test_rating_df['MovieID'].map(movieid_mapper)

  users_df = users_df.sort_values(by=user_id_col).reset_index(drop=True)
  movies_df = movies_df.sort_values(by=movie_id_col).reset_index(drop=True)
  train_rating_df = train_rating_df.sort_values(by=[user_id_col, movie_id_col]).reset_index(drop=True)
  validation_rating_df = validation_rating_df.sort_values(by=[user_id_col, movie_id_col]).reset_index(drop=True)
  test_rating_df = test_rating_df.sort_values(by=[user_id_col, movie_id_col]).reset_index(drop=True)
  return users_df, movies_df, train_rating_df, validation_rating_df, test_rating_df

users_df, movies_df, train_rating_df, validation_rating_df, test_rating_df = finalize_dfs(users_df, movies_df,
                                                                                          train_rating_df, validation_rating_df,
                                                                                          test_rating_df)

In [27]:
train_rating_df

,UserID,MovieID,Rating,Timestamp
0,0,722,5,888732909
1,0,952,3,889751712
2,0,1102,5,889751711
3,0,1340,2,887431672
4,0,1341,5,888732827
...,...,...,...,...
39994,942,1020,4,888638897
39995,942,1025,4,888639042
39996,942,1223,4,888693104
39997,942,1266,2,888639351


In [28]:
def extract_user_feature(users_df):
  scaler = StandardScaler()
  age = scaler.fit_transform(users_df[['Age']])
  
  encoder = OneHotEncoder(handle_unknown='ignore')
  categorical_df = encoder.fit_transform(users_df[['Occupation', 'Gender']]).toarray()
  features = np.hstack((categorical_df, age))
  return features

def extract_movie_feature(movies_df, exclude_cols):
  movies_df = movies_df.drop(exclude_cols, axis=1)
  scaler = StandardScaler()
  movies_df[['Release Year', 'Release Month', 'Average Rating', 'Release Day']] = scaler.fit_transform(
      movies_df[['Release Year', 'Release Month', 'Average Rating', 'Release Day']])
  features = movies_df.to_numpy()
  return features

In [29]:
user_features = extract_user_feature(users_df)
movie_features = extract_movie_feature(movies_df, ['MovieID','Movie Title', 'Release Date'])
movie_features.shape, user_features.shape

((1681, 23), (943, 24))

In [30]:
pca = PCA(0.95)
movie_features_reduced_pca = pca.fit_transform(movie_features)
user_features_reduced_pca = pca.fit_transform(user_features)
movie_features_reduced_pca.shape, user_features_reduced_pca.shape

((1681, 11), (943, 14))

In [31]:
def build_AE(encoding_dim = 8, input_shape= 24):

  input_dim = Input(shape = (input_shape, ))

  encoded1 = Dense(16, activation = 'relu')(input_dim)
  encoded2 = Dense(encoding_dim, activation = 'relu')(encoded1)

  decoded1 = Dense(16, activation = 'relu')(encoded2)
  decoded2 = Dense(input_shape, activation = 'relu')(decoded1)


  autoencoder = Model(inputs = input_dim, outputs = decoded2)
  encoder = Model(inputs = input_dim, outputs = encoded2)

  autoencoder.compile(optimizer = keras.optimizers.Adadelta(learning_rate=0.05), loss = 'binary_crossentropy')
  print(autoencoder.summary())
  return autoencoder, encoder

In [32]:
def train_reduce(AE, encoder, features, epochs=50, batch_size=16):
  print('---------------------------------------Trainig of AE Sarts:')
  AE.fit(features, features, epochs=epochs, batch_size=batch_size, shuffle = True, verbose=0)
  print('--------------------------------------- Dimensionality Reduction Sarts:')
  return encoder.predict(features)


In [33]:
user_autoencoder, user_encoder = build_AE(encoding_dim = 8, input_shape= user_features.shape[-1])
movie_autoencoder, movie_encoder = build_AE(encoding_dim = 8, input_shape= movie_features.shape[-1])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24)]              0         
                                                                 
 dense (Dense)               (None, 16)                400       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 16)                144       
                                                                 
 dense_3 (Dense)             (None, 24)                408       
                                                                 
Total params: 1,088
Trainable params: 1,088
Non-trainable params: 0
_________________________________________________________________
None
Model: "model_2"
_____________________________________

In [34]:
user_features_reduced_ae = train_reduce(user_autoencoder, user_encoder, user_features)
movie_features_reduced_ae = train_reduce(movie_autoencoder, movie_encoder, movie_features)
user_features_reduced_ae.shape, movie_features_reduced_ae.shape

---------------------------------------Trainig of AE Sarts:
--------------------------------------- Dimensionality Reduction Sarts:
30/30 [==============================] - 0s 2ms/step
---------------------------------------Trainig of AE Sarts:
--------------------------------------- Dimensionality Reduction Sarts:
53/53 [==============================] - 0s 1ms/step


((943, 8), (1681, 8))

In [35]:
def generate_edge(rating_df, rating_threshold=4):

  graph_edges = [[],[]]
  edge_weight = []

  for userID, movieID, rating in rating_df[['UserID','MovieID','Rating']].itertuples(index=False):
    if rating >= rating_threshold:
      graph_edges[0].append(userID)
      graph_edges[1].append(movieID)
      edge_weight.append(rating)
    
    else:
      continue
  
  return torch.tensor(graph_edges, dtype=torch.long), torch.tensor(edge_weight, dtype=torch.float)

In [36]:
graph_edges_train, label_train = generate_edge(train_rating_df, rating_threshold=0)
graph_edges_validation, label_validation = generate_edge(validation_rating_df, rating_threshold=0)
graph_edges_test, label_test = generate_edge(test_rating_df, rating_threshold=0)


In [39]:
def generate_hetero_dataset(data, movie_feature, user_feature, edges, labels):
  dataset = HeteroData()

  dataset["user"].node_id = torch.tensor(users_df['UserID'].unique().astype(int), dtype=torch.int)
  dataset["movie"].node_id = torch.tensor(movies_df['MovieID'].unique().astype(int), dtype=torch.int)
  dataset["movie"].x = torch.tensor(movie_feature, dtype=torch.float)
  dataset["user"].x = torch.tensor(user_feature, dtype=torch.float)
  dataset["user", "rates", "movie"].edge_index = edges
  dataset["user", "rates", "movie"].edge_label_index = edges
  dataset["user", "rates", "movie"].edge_label = labels

  dataset = transforms.ToUndirected()(dataset)
  dataset = transforms.NormalizeFeatures()(dataset)
  return dataset

In [40]:
train_dataset = generate_hetero_dataset(train_rating_df, movie_features_reduced_ae, user_features_reduced_ae, graph_edges_train, label_train)
validation_dataset = generate_hetero_dataset(validation_rating_df, movie_features_reduced_ae, user_features_reduced_ae, graph_edges_validation, label_validation)
test_dataset = generate_hetero_dataset(test_rating_df, movie_features_reduced_ae, user_features_reduced_ae, graph_edges_test, label_test)

In [41]:
(validation_dataset["user", "rates", "movie"].edge_index).max(), validation_dataset["movie"].x.size(0)

(tensor(1680), 1681)

In [48]:
def generate_data_loader(dataset):
  edge_label_index = dataset["user", "rates", "movie"].edge_label_index
  edge_label = dataset["user", "rates", "movie"].edge_label
  data_loader = LinkNeighborLoader(
      data=dataset,
      num_neighbors=[20, 10],
      neg_sampling_ratio=2.0,
      edge_label_index=(("user", "rates", "movie"), edge_label_index),
      edge_label=edge_label,
      batch_size=32,
      shuffle=True
      )
  return data_loader

In [49]:
train_loader = generate_data_loader(train_dataset)
validation_loader = generate_data_loader(validation_dataset)
test_loader = generate_data_loader(test_dataset)

In [78]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        return x



class Classifier(torch.nn.Module):
    def forward(self, x_user, x_movie, edge_label_index):

        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        return (edge_feat_user * edge_feat_movie).sum(dim=-1)

class Model(torch.nn.Module):
    
    def __init__(self, data, hidden_channels, user_features_dim, movie_features_dim):
        super().__init__()


        self.user_lin = torch.nn.Linear(user_features_dim, hidden_channels)
        self.movie_lin = torch.nn.Linear(movie_features_dim, hidden_channels)
        self.user_emb = torch.nn.Embedding(data["user"].num_nodes, hidden_channels)
        self.movie_emb = torch.nn.Embedding(data["movie"].num_nodes, hidden_channels)

        self.gnn = GNN(hidden_channels)

        self.gnn = to_hetero(self.gnn, metadata=data.metadata())
        self.classifier = Classifier()
    
    def forward(self, data):
        x_dict = {
          "user": self.user_lin(data["user"].x) + self.user_emb(data["user"].node_id),
          "movie": self.movie_lin(data["movie"].x) + self.movie_emb(data["movie"].node_id),
        } 

        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["movie"],
            data["user", "rates", "movie"].edge_label_index,
        )
        return pred
        
model = Model(data=train_dataset , hidden_channels=16, user_features_dim=8, movie_features_dim=8)
model

Model(
  (user_lin): Linear(in_features=8, out_features=16, bias=True)
  (movie_lin): Linear(in_features=8, out_features=16, bias=True)
  (user_emb): Embedding(943, 16)
  (movie_emb): Embedding(1681, 16)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (user__rates__movie): SAGEConv(16, 16, aggr=mean)
      (movie__rev_rates__user): SAGEConv(16, 16, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__rates__movie): SAGEConv(16, 16, aggr=mean)
      (movie__rev_rates__user): SAGEConv(16, 16, aggr=mean)
    )
    (conv3): ModuleDict(
      (user__rates__movie): SAGEConv(16, 16, aggr=mean)
      (movie__rev_rates__user): SAGEConv(16, 16, aggr=mean)
    )
  )
  (classifier): Classifier()
)

In [80]:
# del user_autoencoder, user_encoder, movie_autoencoder, movie_encoder, pca

In [81]:
class Learning_Evaluation(torch.nn.Module):
    def __init__(
        self,
        model,
        learning_rate=0.01,
        best_results=[np.Inf, np.Inf, np.Inf, np.Inf, np.Inf],
        ):
      
      super().__init__()
      self.model = model
      self.optimizer = torch.optim.SGD(self.model.parameters(), lr=learning_rate)
      self.criterion = torch.nn.MSELoss()
      self.best_results = best_results
    
    def train(self, data_loader):
        self.model.train()

        for data in data_loader:
            out = self.model(data)
            label = data["user", "rates", "movie"].edge_label
            loss = self.criterion(out, label)
            # loss = F.mse_loss(pred, ground_truth)
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

    # def evaluate(self, data_loader):
    #     self.model.eval()
        
    #     predicted = []
    #     labels = []
    #     total_loss = total_sample = 0
        
    #     for data in data_loader:
    #       with torch.no_grad():
    #         data.to(device)
    #         out = model(data)
    #         label = data["user", "rates", "movie"].edge_label
    #         total_loss += self.criterion(out, label) * len(out)
    #         # total_loss += F.binary_cross_entropy_with_logits(out, label)
    #         total_sample += len(out)
    #         predicted.append(out)
    #         labels.append(label)
            
    #     predicted = torch.cat(predicted, dim=0).cpu().numpy()
    #     labels = torch.cat(labels, dim=0).cpu().numpy()
    #     # auc = roc_auc_score(labels, predicted)

    #     return total_loss/ total_sample

    def train_model(self, train_data_loader, validation_data_loader, n_epochs=15, best_model_saving_path='best_model.pth'):
      
      list_train_mse, list_train_rmse, list_validation_rmse, list_validation_mse = [], [], [], []
      print('Train and Evaluation started...')
      for epoch in range(1, n_epochs+1):
          self.train(train_data_loader)
          
          # train_auc, train_loss = self.evaluate(train_data_loader)
          _ , _ , train_rmse_loss, train_mse_loss = self.evaluate(train_data_loader)
          list_train_mse.append(train_mse_loss)
          list_train_rmse.append(train_rmse_loss)
          
          _ , _ , validation_rmse_loss, validation_mse_loss = self.evaluate(validation_data_loader)
          list_validation_mse.append(validation_mse_loss)
          list_validation_rmse.append(validation_rmse_loss)
          
          if self.best_results[-2] > validation_mse_loss :
            self.best_results[0] = epoch
            self.best_results[1], self.best_results[2] = train_mse_loss, train_rmse_loss
            self.best_results[-2], self.best_results[-1] = validation_mse_loss, validation_rmse_loss
            torch.save(model, best_model_saving_path)


          # print(f'Epoch: {epoch:03d}, Train ROC-AUC: {train_auc:.4f}, Train Loss: {train_loss:.4f}, Validation ROC-AUC: {validation_auc:.4f}, Validation Loss: {validation_loss:.4f}')
          print(f'Epoch: {epoch:03d}, Train MSE: {train_mse_loss:.4f}, Train RMSE: {train_rmse_loss:.4f}, Validation MSE: {validation_mse_loss:.4f}, Validation RMSE: {validation_rmse_loss:.4f}')
      
      print('---------------------------------------------------')
      print('Train and Evaluation finished...')
      print(f'Best Results of the model : Epoch: {self.best_results[0]:03d}, Train RMSE: {self.best_results[2]:.4f}, Validation RMSE: {self.best_results[-1]:.4f}')
      print(f'Model weights restored from epoch: {self.best_results[0]:03d}')
      return list_train_mse, list_train_rmse, list_validation_rmse, list_validation_mse

    def evaluate(self, data_loder, best_model_path=None):
      self.model.eval()
      
      if not best_model_path is None:
        model = torch.load(best_model_path)
      else:
        model = self.model
      predicted = []
      labels = []
      for data in data_loder:
          with torch.no_grad():
              data.to(device)
              predicted.append(model(data))
              labels.append(data["user", "rates", "movie"].edge_label)

      predicted = torch.cat(predicted, dim=0).cpu().numpy()
      labels = torch.cat(labels, dim=0).cpu().numpy()
      rmse = mean_squared_error(labels, predicted, squared=False)
      mse = mean_squared_error(labels, predicted, squared=True)

      return labels, predicted, rmse, mse

      
      

In [ ]:
experiment = Learning_Evaluation(model)
list_train_mse, list_train_rmse, list_validation_rmse, list_validation_mse = experiment.train_model(train_loader, validation_loader, n_epochs=50)

Train and Evaluation started...
Epoch: 001, Train MSE: 2.2699, Train RMSE: 1.5066, Validation MSE: 2.8111, Validation RMSE: 1.6766
Epoch: 002, Train MSE: 2.0781, Train RMSE: 1.4415, Validation MSE: 2.6181, Validation RMSE: 1.6181
Epoch: 003, Train MSE: 1.9958, Train RMSE: 1.4127, Validation MSE: 2.5929, Validation RMSE: 1.6102
Epoch: 004, Train MSE: 1.9353, Train RMSE: 1.3911, Validation MSE: 2.5552, Validation RMSE: 1.5985
Epoch: 005, Train MSE: 1.8701, Train RMSE: 1.3675, Validation MSE: 2.4814, Validation RMSE: 1.5752
Epoch: 006, Train MSE: 1.8709, Train RMSE: 1.3678, Validation MSE: 2.5622, Validation RMSE: 1.6007
Epoch: 007, Train MSE: 1.8219, Train RMSE: 1.3498, Validation MSE: 2.4754, Validation RMSE: 1.5733
Epoch: 008, Train MSE: 1.7727, Train RMSE: 1.3314, Validation MSE: 2.4825, Validation RMSE: 1.5756


In [72]:
experiment.evaluate(test_loader)

(array([3., 3., 3., ..., 0., 0., 0.], dtype=float32),
 array([7.7040493e-01, 4.6561127e+00, 2.1630859e+00, ..., 4.2936397e-01,
        8.3954275e-02, 2.9045194e-03], dtype=float32),
 1.3860945,
 1.9212579)

In [52]:
import tqdm
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1, 5):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        sampled_data.to(device)
        pred = model(sampled_data)
        ground_truth = sampled_data["user", "rates", "movie"].edge_label
        loss = F.mse_loss(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cpu'


100%|██████████| 1250/1250 [00:27<00:00, 46.25it/s]


Epoch: 001, Loss: 2.6201


100%|██████████| 1250/1250 [00:15<00:00, 83.12it/s]


Epoch: 002, Loss: 2.3464


100%|██████████| 1250/1250 [00:16<00:00, 74.96it/s]


Epoch: 003, Loss: 2.2599


100%|██████████| 1250/1250 [00:15<00:00, 81.83it/s]

Epoch: 004, Loss: 2.2124


In [53]:
predicted = []
labels = []
i=0
for sampled_data in tqdm.tqdm(train_loader):
    with torch.no_grad():
        sampled_data.to(device)
        predicted.append(model(sampled_data))
        labels.append(sampled_data["user", "rates", "movie"].edge_label)

predicted = torch.cat(predicted, dim=0).cpu().numpy()
labels = torch.cat(labels, dim=0).cpu().numpy()
# auc = roc_auc_score(labels, predicted)
# predicted, labels
print()
# print(f"Test AUC: {auc:.4f}")

100%|██████████| 1250/1250 [00:08<00:00, 141.50it/s]

In [54]:
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.metrics import classification_report

mean_squared_error(labels, predicted, squared=False), mean_squared_error(labels, predicted, squared=True)


(1.476913, 2.1812718)

In [56]:
l = np.where(labels>=3, 1, 0)
p = np.where(predicted>=3, 1, 0)
print(classification_report(l, p))

              precision    recall  f1-score   support

           0       0.74      0.99      0.85     87311
           1       0.71      0.05      0.09     32686

    accuracy                           0.74    119997
   macro avg       0.72      0.52      0.47    119997
weighted avg       0.73      0.74      0.64    119997



In [ ]:
predicted = []
labels = []
i=0
for sampled_data in tqdm.tqdm(validation_loader):
    with torch.no_grad():
        sampled_data.to(device)
        predicted.append(model(sampled_data))
        labels.append(sampled_data["user", "rates", "movie"].edge_label)

predicted = torch.cat(predicted, dim=0).cpu().numpy()
labels = torch.cat(labels, dim=0).cpu().numpy()
# auc = roc_auc_score(labels, predicted)
# predicted, labels
print()
# print(f"Test AUC: {auc:.4f}")

  0%|          | 0/625 [00:00<?, ?it/s]


IndexError: ignored

In [ ]:
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.metrics import classification_report

mean_squared_error(labels, predicted, squared=False), mean_squared_error(labels, predicted, squared=True)


ValueError: ignored

In [ ]:
l = np.where(labels>=3, 1, 0)
p = np.where(predicted>=3, 1, 0)
print(classification_report(l, p))